# Fronteras
Primero se definen las fronteras de donde se van a obtener los datos.

En este caso, se van a obtener de madrid Central aproximadamente

In [3]:
import folium

# Definir los límites de Madrid Central
MADRID_CENTRAL_BOUNDS = [
    (40.4312602966105, -3.724047827443745),
    (40.42933043772077, -3.6871072435491214),
    (40.40546217601727, -3.6894655434080916),
    (40.40398039415917, -3.702653697350822),
    (40.40744199716853, -3.7200607518707085),
    (40.422406945606085, -3.716777655227359)
]


# Calcular el centro de Madrid Central
center = [(MADRID_CENTRAL_BOUNDS[0][0] + MADRID_CENTRAL_BOUNDS[1][0]) / 2, (MADRID_CENTRAL_BOUNDS[0][1] + MADRID_CENTRAL_BOUNDS[1][1]) / 2]

# Crear un mapa centrado en el centro de Madrid Central
m = folium.Map(location=center, zoom_start=15)

# Agregar un polígono que representa los límites de Madrid Central
folium.Polygon(locations=MADRID_CENTRAL_BOUNDS, color='red', fill_opacity=0.1).add_to(m)

# Mostrar el mapa
m


# Obtener restaurantes

Ahora se busca obtener de la API de google maps, los datos de restaurantes

Esto es una primera muestra para ver los datos que se pueden obtener

In [4]:
import requests
import time

# Leer la clave API desde un archivo de texto en el escritorio
with open(r"C:\Users\empca\Desktop\API_Google_Maps.txt") as f:
    API_KEY = f.read().strip()

# Definir las coordenadas de la zona a buscar
BOUNDARY_COORDS = [
    (40.4312602966105, -3.724047827443745),
    (40.42933043772077, -3.6871072435491214),
    (40.40546217601727, -3.6894655434080916),
    (40.40398039415917, -3.702653697350822),
    (40.40744199716853, -3.7200607518707085),
    (40.422406945606085, -3.716777655227359)
]

# Formatear las coordenadas como una cadena de texto para la API
boundary_str = '|'.join([f'{lat},{lng}' for lat, lng in BOUNDARY_COORDS])
boundary_str = f'({boundary_str})'

# Definir la URL base para la API de Google Places
URL_BASE = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

# Definir los parámetros para la solicitud
params = {
    'query': 'restaurant',
    'bounds': boundary_str,
    'key': API_KEY
}

# Realizar la solicitud a la API de Google Places
response = requests.get(URL_BASE, params=params)

# Obtener la información de los restaurantes
restaurant_data = response.json()['results']

# Obtener el número total de resultados
print(f"Número de restaurantes: {len(restaurant_data)}")

# Iterar a través de cada restaurante y obtener su nombre, dirección, coordenadas, horario, teléfono y precio medio
for restaurant in restaurant_data:
    name = restaurant['name']
    address = restaurant['formatted_address']
    tipo = restaurant['types']
    lat = restaurant['geometry']['location']['lat']
    lng = restaurant['geometry']['location']['lng']
    rating = restaurant.get('rating', 'N/A')
    num_ratings = restaurant.get('user_ratings_total', 'N/A')
    place_id = restaurant['place_id']

    # Obtener el horario del restaurante
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
    details_params = {
        'place_id': place_id,
        'fields': 'opening_hours,formatted_phone_number,price_level',
        'key': API_KEY
    }
    details_response = requests.get(details_url, params=details_params)
    details_data = details_response.json().get('result', {})
    opening_hours = details_data.get('opening_hours', {}).get('weekday_text', [])
    phone_number = details_data.get('formatted_phone_number', 'N/A')
    price_level = details_data.get('price_level', 'N/A')

    # Imprimir la información del restaurante
    print(f"Restaurante: {name}")
    print(f"Dirección: {address}")
    print(f"Latitud: {lat}, Longitud: {lng}")
    print(f"Tipo: {tipo}")
    print(f"Calificación: {rating}")
    print(f"Número de reseñas: {num_ratings}")
    print(f"Teléfono: {phone_number}")
    print(f"Precio medio aproximado: {price_level}")

    print("Horario:")
    for day in opening_hours:
        print(day)

    print()
    
    # Hacer una pausa de 1 segundo entre cada solicitud para evitar superar el límite de la API
    time.sleep(1)
    
    break

Número de restaurantes: 20
Restaurante: Urban Bistro
Dirección: Av. General, 43b, 28042 Madrid, Spain
Latitud: 40.47103300000001, Longitud: -3.5799579
Tipo: ['restaurant', 'food', 'point_of_interest', 'establishment']
Calificación: 4.4
Número de reseñas: 65
Teléfono: 645 68 45 73
Precio medio aproximado: N/A
Horario:
Monday: 5:00 – 11:30 PM
Tuesday: 5:00 – 11:30 PM
Wednesday: 5:00 – 11:30 PM
Thursday: 5:00 – 11:30 PM
Friday: 5:00 – 11:30 PM
Saturday: 5:00 – 11:30 PM
Sunday: 5:00 – 11:30 PM



Ahora se modifica el código para que los resultados se guarden en un Dataframe para después poder exportarlos

In [11]:
import requests
import time
import pandas as pd

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")

# Leer la clave API desde un archivo de texto en el escritorio
with open(r"C:\Users\empca\Desktop\API_Google_Maps.txt") as f:
    API_KEY = f.read().strip()

# Definir las coordenadas de la zona a buscar
BOUNDARY_COORDS = [
    (40.4312602966105, -3.724047827443745),
    (40.42933043772077, -3.6871072435491214),
    (40.40546217601727, -3.6894655434080916),
    (40.40398039415917, -3.702653697350822),
    (40.40744199716853, -3.7200607518707085),
    (40.422406945606085, -3.716777655227359)
]

# Formatear las coordenadas como una cadena de texto para la API
boundary_str = '|'.join([f'{lat},{lng}' for lat, lng in BOUNDARY_COORDS])
boundary_str = f'({boundary_str})'


# Definir la URL base para la API de Google Places
URL_BASE = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

# Definir los parámetros para la solicitud
params = {
    'query': 'supermarket',
    'bounds': boundary_str,
    'key': API_KEY
}

# Inicializar el dataframe de resultados
df = pd.DataFrame(columns=['name', 'address', 'lat', 'lng', 'rating', 'num_ratings', 'place_id', 'opening_hours', 'phone_number', 'price_level'])

# Realizar varias solicitudes para obtener más resultados
for i in range(6):
    # Realizar la solicitud a la API de Google Places
    response = requests.get(URL_BASE, params=params)

    # Obtener la información de los restaurantes de la página actual
    current_data = response.json().get('results', [])

    # Iterar a través de cada restaurante y obtener su nombre, dirección y coordenadas
    for restaurant in current_data:
        lat = restaurant['geometry']['location']['lat']
        lng = restaurant['geometry']['location']['lng']

        location = geolocator.reverse(f"{lat},{lng}")
        address = location.raw.get('address', {})
        postal_code = address.get('postcode', 'N/A')

        restaurant_dict = {
            'name': restaurant['name'],
            'address': restaurant['formatted_address'],
            'lat': lat,
            'lng': lng,
            'rating': restaurant.get('rating', 'N/A'),
            'num_ratings': restaurant.get('user_ratings_total', 'N/A'),
            'place_id': restaurant['place_id'],
            'postal_code': postal_code,
            "tipo": "supermercado"
        }

        # Obtener el horario del restaurante
        details_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
        details_params = {
            'place_id': restaurant_dict['place_id'],
            'fields': 'opening_hours,formatted_phone_number,price_level',
            'key': API_KEY
        }
        details_response = requests.get(details_url, params=details_params)
        details_data = details_response.json().get('result', {})
        restaurant_dict['opening_hours'] = details_data.get('opening_hours', {}).get('weekday_text', [])
        restaurant_dict['phone_number'] = details_data.get('formatted_phone_number', 'N/A')
        restaurant_dict['price_level'] = details_data.get('price_level', 'N/A')

        # Agregar el restaurante al dataframe
        df = pd.concat([df, pd.DataFrame([restaurant_dict], index=[0])], ignore_index=True)

    # Obtener el token de la página siguiente
    next_page_token = response.json().get('next_page_token')

    # Si no hay token de la página siguiente, detener la iteración
    if not next_page_token:
        break

    # Hacer una pausa de 2 segundos antes de solicitar la siguiente página
    time.sleep(2)

    # Actualizar los parámetros de la solicitud para obtener la página siguiente
    params['pagetoken'] = next_page_token

    # Si se han obtenido 200 restaurantes, detener la iteración
    if len(df) >= 200:
        break

# Imprimir el número total de restaurantes obtenidos
print(f"Se encontraron {len(df)} restaurantes en la zona especificada.")

   




Se encontraron 60 restaurantes en la zona especificada.


In [13]:
# Exporta el DataFrame a un archivo CSV
df.to_csv('supermercados_tripulaciones.csv', index=False)   

In [12]:
df

,name,address,lat,lng,rating,num_ratings,place_id,opening_hours,phone_number,price_level,postal_code,tipo
0,Supermercados Dia,"C. de López de Aranda, 37, 28027 Madrid, Spain",40.444719,-3.634218,3.4,90,ChIJoZlFEHsvQg0RFTzyo7JxIkA,"[Monday: 9:00 AM – 9:30 PM, Tuesday: 9:00 AM –...",912 17 04 53,N/A,28027,supermercado
1,Supercor Exprés,"C. de Alcalá, 494, 28027 Madrid, Spain",40.440022,-3.62879,3.6,174,ChIJbYUsLAwvQg0RJeKkd5_G4e8,"[Monday: 8:30 AM – 12:00 AM, Tuesday: 8:30 AM ...",913 75 79 68,N/A,28027,supermercado
2,Dia,"C. de Alcalá, 451, 28027 Madrid, Spain",40.438995,-3.634957,3.9,541,ChIJB0rSsBMvQg0RdecP8jZteTM,"[Monday: 8:30 AM – 10:00 PM, Tuesday: 8:30 AM ...",912 17 04 53,N/A,28027,supermercado
3,Carrefour Express EESS,"Gta. de Yucatán, 28042 Madrid, Spain",40.450768,-3.627143,4,278,ChIJ_YnIsgYvQg0RExizEczxbQM,"[Monday: 9:00 AM – 9:00 PM, Tuesday: 9:00 AM –...",914 90 89 00,N/A,28043,supermercado
4,Tienda Supercor Stop & Go,"C. de Arturo Soria, 29, 28027 Madrid, Spain",40.441306,-3.641001,5,1,ChIJfXwPMCQvQg0RyQrSIVD4HtQ,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",N/A,N/A,28027,supermercado
5,Autoservicio Peraleda,"Tr.ª de López de Aranda, 3, 28027 Madrid, Spain",40.442527,-3.63692,5,2,ChIJT7i3LBEvQg0R67aZ8XW3GfA,"[Monday: 9:00 AM – 3:00 PM, 5:00 – 10:00 PM, T...",N/A,N/A,28027,supermercado
6,"Alimentacion, pan, bebidas, bazar","C. Germán Pérez Carrasco, 85, 28027 Madrid, Spain",40.438096,-3.639869,4.8,5,ChIJOe5Nd54vQg0RyUyUpX2Qyr8,"[Monday: 9:00 AM – 12:00 AM, Tuesday: 9:00 AM ...",613 18 54 25,N/A,28027,supermercado
7,Chino,"Pl. de Ensidesa, 3, 28027 Madrid, Spain",40.443531,-3.622899,5,7,ChIJhyT-NAgvQg0ReDjMF-D8Ssc,"[Monday: 9:00 AM – 11:00 PM, Tuesday: 9:00 AM ...",N/A,N/A,28027,supermercado
8,Hiper 100 y Mas,"28027 Madrid, Spain",40.438602,-3.634941,4,2,ChIJQ2YusBMvQg0RjcDneSLE3xA,[],N/A,N/A,28027,supermercado
9,Al Madina Frutas - Verduras Y Alimentación,"28027 Madrid, Spain",40.438363,-3.635746,0,0,ChIJc8GemBMvQg0Ri4prIiFfZqI,"[Monday: 9:00 AM – 2:30 PM, 5:00 – 8:00 PM, Tu...",N/A,N/A,28027,supermercado


In [20]:
import requests

response = requests.get('https://jsonplaceholder.typicode.com/todos')
if response.status_code == 200:
    orders = response.json()
    for order in orders:
        if order['completed']:
            print(f"El pedido con id {order['id']} ha sido entregado por el usuario {order['userId']}.")
        else:
            print(f"El pedido con id {order['id']} ha sido solicitado por el usuario {order['userId']}.")
else:
    print('Error al obtener los pedidos: ', response.content)

El pedido con id 1 ha sido solicitado por el usuario 1.
El pedido con id 2 ha sido solicitado por el usuario 1.
El pedido con id 3 ha sido solicitado por el usuario 1.
El pedido con id 4 ha sido entregado por el usuario 1.
El pedido con id 5 ha sido solicitado por el usuario 1.
El pedido con id 6 ha sido solicitado por el usuario 1.
El pedido con id 7 ha sido solicitado por el usuario 1.
El pedido con id 8 ha sido entregado por el usuario 1.
El pedido con id 9 ha sido solicitado por el usuario 1.
El pedido con id 10 ha sido entregado por el usuario 1.
El pedido con id 11 ha sido entregado por el usuario 1.
El pedido con id 12 ha sido entregado por el usuario 1.
El pedido con id 13 ha sido solicitado por el usuario 1.
El pedido con id 14 ha sido entregado por el usuario 1.
El pedido con id 15 ha sido entregado por el usuario 1.
El pedido con id 16 ha sido entregado por el usuario 1.
El pedido con id 17 ha sido entregado por el usuario 1.
El pedido con id 18 ha sido solicitado por el usu

In [14]:
import requests

response = requests.get('https://fakestoreapi.com/orders')
orders = response.json()

for order in orders:
    print(order)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)